In [1]:
from algorithms import *

In [ ]:
import numpy as np
import pandas as pd
import time

from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


def test_solvers():
    X_syn, y_syn = make_classification(
        n_samples=1000, n_features=100, 
        n_informative=50, n_redundant=2,
        n_classes=2, random_state=42, class_sep=2.0,
    )

    y_syn = 2 * y_syn - 1
    data = load_breast_cancer()
    X_real, y_real = data.data, data.target

    scaler = StandardScaler()
    X_syn = scaler.fit_transform(X_syn)
    X_real = scaler.fit_transform(X_real)

    X_syn_train, X_syn_test, y_syn_train, y_syn_test = train_test_split(
        X_syn, y_syn, test_size=0.3, random_state=1)
    X_real_train, X_real_test, y_real_train, y_real_test = train_test_split(
        X_real, y_real, test_size=0.3, random_state=1)

    solvers = ['cg', 'lbfgs']
    results = []

    for solver in solvers:
        clf = SquaredHingeClassifier(C=1.0, solver=solver)


        #synthetic data
        t0 = time.time()
        clf.fit(X_syn_train, y_syn_train)
        t_syn = time.time() - t0
        y_syn_pred = clf.predict(X_syn_test)
        acc_syn = accuracy_score(y_syn_test, y_syn_pred)

        #real data
        t0 = time.time()
        clf.fit(X_real_train, y_real_train)
        t_real = time.time() - t0
        y_real_pred = clf.predict(X_real_test)
        acc_real = accuracy_score(y_real_test, y_real_pred)

        results.append({
            'solver': solver,
            'syn_accuracy': acc_syn,
            'syn_time_s': t_syn,
            'real_accuracy': acc_real,
            'real_time_s': t_real
        })

    df = pd.DataFrame(results)
    print(df)
    print("\nSynthetic Data ")
    for solver in solvers:
        clf = SquaredHingeClassifier(C=1.0, solver=solver)
        clf.fit(X_syn_train, y_syn_train)
        print(f"\nSolver = {solver}")
        print(classification_report(y_syn_test, clf.predict(X_syn_test)))
    
    print("\nReal Data:")
    for solver in solvers:
        clf = SquaredHingeClassifier(C=1.0, solver=solver)
        clf.fit(X_real_train, y_real_train)
        print(f"\nSolver = {solver}")
        print(classification_report(y_real_test, clf.predict(X_real_test)))

if __name__ == "__main__":
    test_solvers()


Starting Conjugate Gradient optimization...
Initial gradient norm: 1.099863e+03
Iter   0: loss = 6.219922e+02, grad_norm = 3.754382e+02, alpha = 2.786992e-03
Iter  10: loss = 3.603968e+02, grad_norm = 3.089551e+01, alpha = 1.097096e-02
Iter  20: loss = 3.504162e+02, grad_norm = 1.682805e+01, alpha = 1.628162e-02
Iter  30: loss = 3.474951e+02, grad_norm = 6.700162e+00, alpha = 1.461576e-02
Iter  40: loss = 3.456140e+02, grad_norm = 4.880370e+00, alpha = 8.886498e-03
Iter  50: loss = 3.452259e+02, grad_norm = 3.269530e+00, alpha = 4.973870e-03
Iter  60: loss = 3.451422e+02, grad_norm = 2.026267e+00, alpha = 1.936826e-03
Iter  70: loss = 3.451041e+02, grad_norm = 1.458859e+00, alpha = 3.829472e-03
Iter  80: loss = 3.450871e+02, grad_norm = 1.243321e+00, alpha = 2.633839e-03
Iter  90: loss = 3.450713e+02, grad_norm = 6.476348e-01, alpha = 3.256511e-03
Iter 100: loss = 3.450669e+02, grad_norm = 4.684044e-01, alpha = 3.813910e-03
Iter 110: loss = 3.450645e+02, grad_norm = 2.673619e-01, alpha

c:\Users\hucu\Desktop\Optimization\algorithms.py:219: RuntimeWarning: CG did not converge. Final gradient norm: 2.259978e-03 > 1e-06
  warnings.warn(
c:\Users\hucu\Desktop\Optimization\algorithms.py:82: LineSearchWarning: The line search algorithm did not converge
  result = line_search(
c:\Users\hucu\Desktop\Optimization\algorithms.py:219: RuntimeWarning: CG did not converge. Final gradient norm: 2.259978e-03 > 1e-06
  warnings.warn(
c:\Users\hucu\Desktop\Optimization\algorithms.py:82: LineSearchWarning: The line search algorithm did not converge
  result = line_search(
